In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Cartographie

On va regarder comment dessiner une carte géographique avec de l'information par dessus.

_Note : si les cartes ne sont pas visibles à l'ouverture de la feuille, veuillez recalculer toutes les cellules avec `Cell -> Run all`._

## Les données

On récupère les prix du jour de l'essence dans les stations de France sur
https://www.prix-carburants.gouv.fr/
(voir le programme <a href="data/prix_carburants.py">prix_carburants.py</a> qui
fait le travail).

In [2]:
from data import prix_carburants
df = prix_carburants.get_gaz_price()
df.iloc[0]

adresse      596 AVENUE DE TREVOUX
cp                           01000
ville        SAINT-DENIS-LèS-BOURG
latitude                   46.2011
longitude                  5.19791
essence                     Gazole
prix                         1.444
maj            2018-09-12 15:33:01
Name: 0, dtype: object

Récupérons les 10 meilleurs prix et les 10 pires pour l'essence sans plomb 95 européenne (E10). Attention certaines stations ne mettent pas à jour leur prix ce qui fait qu'on peut avoir des prix d'il y a un an, donc totalement faux !

In [3]:
best = df[(df.essence == 'E10') 
          & (df.maj + dt.timedelta(days=7) > dt.datetime.now())].sort_values('prix')
print('Nombre de stations prises en compte : %d' % len(best))
worst = best[-10:]
best = best[:10]

Nombre de stations prises en compte : 3207


## Folium pour dessiner des cartes

[Folium](http://python-visualization.github.io/folium/) permet d'afficher des données sur une carte. Pour cela elle s'appuie sur la bibliothèque Javascript [Leaflet](https://leafletjs.com/) laquelle utilise les cartes de

* [OpenStreetMap](https://www.openstreetmap.org/) par défaut (libre)
* [Staten](http://maps.stamen.com) s'appuie sur OpenStreetMao mais produit un rendu différent
* [Boxmap](https://www.boxmap.com/maps/) (compte gratuit limité possible)

Il est possible d'utiliser les cartes de Google si on a un compte qui le permet avec [FoliumGEE](https://github.com/mccarthyryanc/folium_gee).

In [4]:
import folium

In [5]:
france = folium.Map(location=[46.5,2], zoom_start=5)
#france = folium.Map(location=[46.5,2], zoom_start=5, tiles='Stamen Terrain')

for lat,lon,val in zip(best.latitude, best.longitude, best.prix):
    folium.Marker((lat,lon), popup="%.2f €/l" % val).add_to(france)
for lat,lon,val in zip(worst.latitude, worst.longitude, worst.prix):
    folium.Marker((lat,lon), 
                  icon=folium.Icon(color='red', icon='exclamation-sign'),
                  popup="%.2f €/l" % val
                 ).add_to(france)
vb = best.iloc[0] # very best
folium.CircleMarker((vb.latitude,vb.longitude), radius=20, 
                    popup='Best !', color='green').add_to(france)
france

Bien sûr on peut zoomer et déplacer la carte.

#### Fond de carte

Les principaux fonds de carte sont `OpenStreetMap` (par défaut), `Stamen Terrain`, `Staten Toner`, `Statemen Watercolor`, `Mapbox Bright` et `Mapbox Control`.

#### Icones

Pour les icones possibles voir https://github.com/lvoogdt/Leaflet.awesome-markers

### Objets géographiques

[GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) est un format ouvert pour définir des objets géographique. Folium permet d'afficher directement ces objets [`folium.GeoJson` et `folium.TopoJson`](http://python-visualization.github.io/folium/quickstart.html#GeoJSON/TopoJSON-Overlays) (comme on a utilisé `folium.Marker`). 


Mais
on peut aussi colorier les objets ajoutés en fonction de données dans un tableau.
Ainsi sur [France Geojson](https://github.com/gregoiredavid/france-geojson) on peut récupérer les entités administratives de la France ce qui va nous permettre de faire une carte coloriée avec le prix moyen de l'essence par département.

Pour cela il faut un DataFrame qui ait une valeur par département et donc aussi l'identifiant du département. Dans le fichier récupéré sur France Geojson,
on note que le code des départements est une chaîne de 2 caractère. Il faut donc avoir la même chaine dans notre tableau.

In [6]:
df['département'] = df.cp.apply(lambda x:x[:2])
prix_dep = df[(df.essence == 'E10') & (df.maj + dt.timedelta(days=30) > dt.datetime.now())] \
             [['département','prix']].groupby('département', as_index=False).mean()
display(prix_dep.sort_values('prix').head(2))
display(prix_dep.sort_values('prix').tail(2))

,département,prix
67,79,1.50900
26,31,1.50956


,département,prix
79,92,1.629274
63,75,1.682239


In [7]:
france = folium.Map(location=[46.5,2], zoom_start=5, tiles='Stamen Toner')

france.choropleth(
    geo_data = 'data/departements-version-simplifiee.geojson',
    data = prix_dep,
    columns = ['département', 'prix'],
    key_on = 'feature.properties.code',
    fill_color = 'YlOrRd',
    fill_opacity = 0.6,
    line_opacity = 0.2,
    legend_name = "Prix de l\\'essence E10"
)

folium.LayerControl().add_to(france)
france

Certain département n'ont pas de données (sur la côte Atlantique et en Corse pour un problème de notation (20 pour l'essence, 2A et 2B pour la carte))

{{ PreviousNext("24 -- Dynamic graphics with Plotly -- 3D.ipynb", "40 -- A dashboard with Dash -- Layout.ipynb")}}